<a href="https://colab.research.google.com/github/KidsareBornStars/Study/blob/main/Balls_Classification_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/balls

/content/drive/MyDrive/balls


In [2]:
# Importing Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn 
import torch.utils.data as data
from torchvision import transforms, datasets, models
import torch.optim as optim
from torchsummary import summary

In [4]:
base_dir = "/content/drive/MyDrive/balls"

train_dir = os.path.join(base_dir,"train")
test_dir = os.path.join(base_dir,"test")
valid_dir = os.path.join(base_dir,"valid")

In [72]:
# Data Augmentation을 위하여 RandomRotation, RandomResizedCrop, RandomHorizontalFlip, 적용.
transform = {"train": transforms.Compose([transforms.RandomRotation(45),
                                          transforms.RandomResizedCrop(128),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.RandomVerticalFlip(),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.5,],[0.5,])
                                          ]),
             "test": transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5,],[0.5,]),transforms.Resize(128)])}

In [73]:
train_data = datasets.ImageFolder(root=train_dir, transform = transform["train"])
test_data = datasets.ImageFolder(root=test_dir, transform = transform["test"])
valid_data = datasets.ImageFolder(root=valid_dir, transform = transform["test"])

In [74]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # cpu환경에서의 학습이 느려서 gpu환경에서 학습

In [75]:
device

device(type='cuda', index=0)

In [76]:
train_loader = data.DataLoader(train_data,64,True)
test_loader = data.DataLoader(test_data,64,False)
valid_loader = data.DataLoader(valid_data,64,False)

In [77]:
def train(epoch):
    model.train()

    train_loss = 0
    correct = 0
    for i, (input,label) in enumerate(train_loader):
        input=input.to(device)
        label=label.to(device)
        output = model(input)
        loss = criterion(output,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (output.argmax(dim=1) == label).type(torch.float).sum().item()

        if (i+1)%10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)] \tLoss: {:.3f} \tAccuracy: {:.2f}%'.format(
            epoch+1,
            (i+1) * len(input),
            len(train_loader.dataset),
            100. * (i+1) / len(train_loader),
            loss.item(),
            100. * correct / ((i+1) * len(input))))

In [78]:
def test(data_loader):
    model.eval()

    test_loss =0
    correct = 0
    for input,label in data_loader:
        input,label = input.to(device),label.to(device)
        output = model(input)

        test_loss += criterion(output,label).item()
        correct += (output.argmax(dim=1)==label).type(torch.float).sum().item()

    test_loss /= len(test_loader)
    accuracy  = 100. * correct / len(valid_loader.dataset)

    print(f"\nTest set: Average loss: {test_loss:.2f}, Accuracy: {accuracy:.0f}%\n") 

##Normal CNN

In [79]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,8,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        self.fc = nn.Sequential(
            nn.Linear(8*32*32,32),
            nn.ReLU(),
            nn.Linear(32,26))
    
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        return x

In [80]:
criterion=nn.CrossEntropyLoss()

In [81]:
# Accuracy가 비슷해진 것을 확인할 수 있다
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-3)
for epoch in range(15):
    train(epoch)
    test(valid_loader)

Train Epoch: 1 [640/3051 (21%)] 	Loss: 3.146 	Accuracy: 6.72%
Train Epoch: 1 [1280/3051 (42%)] 	Loss: 3.066 	Accuracy: 8.20%
Train Epoch: 1 [1920/3051 (62%)] 	Loss: 3.047 	Accuracy: 10.36%
Train Epoch: 1 [2560/3051 (83%)] 	Loss: 2.987 	Accuracy: 10.66%

Test set: Average loss: 2.99, Accuracy: 12%

Train Epoch: 2 [640/3051 (21%)] 	Loss: 2.711 	Accuracy: 17.97%
Train Epoch: 2 [1280/3051 (42%)] 	Loss: 2.507 	Accuracy: 21.17%
Train Epoch: 2 [1920/3051 (62%)] 	Loss: 2.432 	Accuracy: 22.55%
Train Epoch: 2 [2560/3051 (83%)] 	Loss: 2.571 	Accuracy: 22.11%

Test set: Average loss: 2.88, Accuracy: 27%

Train Epoch: 3 [640/3051 (21%)] 	Loss: 2.289 	Accuracy: 27.50%
Train Epoch: 3 [1280/3051 (42%)] 	Loss: 2.455 	Accuracy: 26.64%
Train Epoch: 3 [1920/3051 (62%)] 	Loss: 2.382 	Accuracy: 27.60%
Train Epoch: 3 [2560/3051 (83%)] 	Loss: 2.393 	Accuracy: 27.85%

Test set: Average loss: 2.44, Accuracy: 32%

Train Epoch: 4 [640/3051 (21%)] 	Loss: 2.347 	Accuracy: 32.97%
Train Epoch: 4 [1280/3051 (42%)] 	Lo

##Expanding Channels

In [82]:
# 더 많은 레이어를 생성하고 채널을 늘려보았다
class CNN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,64,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(64,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32,16,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2))
        self.fc = nn.Sequential(
            nn.Linear(16*16*16,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,26))
    
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        return x

In [83]:
# 역시 overfitting은 어느 정도 해결된 것으로 보인다
model = CNN2().to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-3)
for epoch in range(15):
    train(epoch)
    test(valid_loader)

Train Epoch: 1 [640/3051 (21%)] 	Loss: 3.246 	Accuracy: 5.31%
Train Epoch: 1 [1280/3051 (42%)] 	Loss: 3.083 	Accuracy: 6.25%
Train Epoch: 1 [1920/3051 (62%)] 	Loss: 3.162 	Accuracy: 7.45%
Train Epoch: 1 [2560/3051 (83%)] 	Loss: 3.073 	Accuracy: 8.79%

Test set: Average loss: 3.21, Accuracy: 10%

Train Epoch: 2 [640/3051 (21%)] 	Loss: 2.848 	Accuracy: 15.31%
Train Epoch: 2 [1280/3051 (42%)] 	Loss: 2.835 	Accuracy: 15.70%
Train Epoch: 2 [1920/3051 (62%)] 	Loss: 2.871 	Accuracy: 15.16%
Train Epoch: 2 [2560/3051 (83%)] 	Loss: 2.775 	Accuracy: 15.43%

Test set: Average loss: 2.98, Accuracy: 22%

Train Epoch: 3 [640/3051 (21%)] 	Loss: 2.372 	Accuracy: 22.50%
Train Epoch: 3 [1280/3051 (42%)] 	Loss: 2.549 	Accuracy: 22.03%
Train Epoch: 3 [1920/3051 (62%)] 	Loss: 2.447 	Accuracy: 22.14%
Train Epoch: 3 [2560/3051 (83%)] 	Loss: 2.558 	Accuracy: 22.77%

Test set: Average loss: 2.76, Accuracy: 25%

Train Epoch: 4 [640/3051 (21%)] 	Loss: 2.380 	Accuracy: 26.25%
Train Epoch: 4 [1280/3051 (42%)] 	Loss

## + BN & DO

In [92]:
class CNN_with_BN_DO(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,64,3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2,2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(64,32,3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2,2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32,16,3,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.MaxPool2d(2,2))
        self.fc = nn.Sequential(
            nn.Linear(16*16*16,32),
            nn.ReLU(),
            nn.Linear(32,26))
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        return x

In [93]:
# Epoch을 늘리면 학습이 잘 될 것 같다.
model = CNN_with_BN_DO().to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-3)
for epoch in range(15):
    train(epoch)
    test(valid_loader)

Train Epoch: 1 [640/3051 (21%)] 	Loss: 2.954 	Accuracy: 10.78%
Train Epoch: 1 [1280/3051 (42%)] 	Loss: 2.695 	Accuracy: 15.08%
Train Epoch: 1 [1920/3051 (62%)] 	Loss: 2.820 	Accuracy: 18.59%
Train Epoch: 1 [2560/3051 (83%)] 	Loss: 3.008 	Accuracy: 19.73%

Test set: Average loss: 3.06, Accuracy: 26%

Train Epoch: 2 [640/3051 (21%)] 	Loss: 2.495 	Accuracy: 31.88%
Train Epoch: 2 [1280/3051 (42%)] 	Loss: 2.380 	Accuracy: 33.12%
Train Epoch: 2 [1920/3051 (62%)] 	Loss: 2.134 	Accuracy: 33.39%
Train Epoch: 2 [2560/3051 (83%)] 	Loss: 2.137 	Accuracy: 33.59%

Test set: Average loss: 2.22, Accuracy: 35%

Train Epoch: 3 [640/3051 (21%)] 	Loss: 2.320 	Accuracy: 37.81%
Train Epoch: 3 [1280/3051 (42%)] 	Loss: 2.203 	Accuracy: 35.39%
Train Epoch: 3 [1920/3051 (62%)] 	Loss: 2.261 	Accuracy: 35.83%
Train Epoch: 3 [2560/3051 (83%)] 	Loss: 2.169 	Accuracy: 36.52%

Test set: Average loss: 2.28, Accuracy: 42%

Train Epoch: 4 [640/3051 (21%)] 	Loss: 1.896 	Accuracy: 38.75%
Train Epoch: 4 [1280/3051 (42%)] 	